In [1]:
import requests
import pandas as pd
import time
import random
from urllib.parse import quote, urlencode
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
import time
import json

# Step 1: Fetch earnings data from Money Control API
url_mc = "https://api.moneycontrol.com/mcapi/v1/earnings/actual-estimate?page=1&limit=100"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://www.moneycontrol.com/"
}

session = requests.Session()
response_mc = session.get(url_mc,headers=headers)
data_mc = response_mc.json()

# Step 2: Create a pandas DataFrame with sc_id, stock name, net profit actual, net profit estimated
rows = []
for item in data_mc['data']['list']:
    sc_id = item[0]
    stock_name = item[1]
    net_profit_actual = None
    net_profit_estimated = None
    for qd in item[9]:
        if qd[0] == "Net Profit":
            net_profit_actual = qd[1]
            net_profit_estimated = qd[2]
            break
    rows.append({
        'sc_id': sc_id,
        'stock_name': stock_name,
        'net_profit_actual': net_profit_actual,
        'net_profit_estimated': net_profit_estimated
    })

df = pd.DataFrame(rows)
print("DataFrame after fetching earnings data:")
print(df)

nse_id_list = []
for sc_id in df['sc_id']:
    nse_url = f"https://priceapi.moneycontrol.com/pricefeed/nse/equitycash/{sc_id}"
    try:
        response_nse = session.get(nse_url, headers=headers)
        nse_data = response_nse.json()
        nse_id = nse_data.get('data', {}).get('NSEID')
    except Exception:
        nse_id = None
    nse_id_list.append(nse_id)
    time.sleep(random.uniform(1, 2))  # Small delay to avoid rate limiting

df['nse_id'] = nse_id_list
print("\nDataFrame after fetching NSE IDs:")
print(df)

DataFrame after fetching earnings data:
   sc_id       stock_name net_profit_actual net_profit_estimated
0    HAL  Hindustan Aeron             3,977                2,665
1     EM    Eicher Motors             1,362                1,238
2    TPC       Tata Power             1,028                  992
3   SC12    Shree Cements               574                  383
4   TP13    Torrent Power             1,060                  448
..   ...              ...               ...                  ...
95    DI      Dabur India               320                  313
96   MRF              MRF               512                  341
97   UBB  United Brewerie                98                   92
98  BT09       APL Apollo               293                  248
99     V           Voltas               241                  259

[100 rows x 4 columns]

DataFrame after fetching NSE IDs:
   sc_id       stock_name net_profit_actual net_profit_estimated      nse_id
0    HAL  Hindustan Aeron             3,977 

In [2]:
# Setup undetected headless Chrome
options = uc.ChromeOptions()
options.headless = False  # Set to False if you want to see the browser window
driver = uc.Chrome(options=options)


try:
    # Step 1: Visit home page to set initial cookies
    driver.get("https://www.nseindia.com/")
    time.sleep(2)

    # Step 2: Visit equity quote page
    driver.get("https://www.nseindia.com/get-quotes/equity?symbol=AIRTEL-RE")
    time.sleep(2)

    nse_url = "https://www.nseindia.com/api/corporate-disclosure-getquote"
    exchdisstime_list = []
    for nse_id in df['nse_id']:
        full_url = f"{nse_url}?symbol={nse_id}&corpType=announcement&market=equities&from_date=01-05-2025&to_date=14-05-2025"
        
        try:
            driver.get(full_url)
            time.sleep(3)

            # Step 4: Get raw JSON text — rendered as a body tag
            body = driver.find_element(By.TAG_NAME, "pre")  # sometimes <pre>, sometimes <body>
            json_text = body.text

            # Step 5: Parse and print JSON
            data_nse = json.loads(json_text)
        except Exception as e:
            print(e)
            data_nse = []
        
        print(full_url)
        print(data_nse)
        # Find the element with desc matching "Outcome of Board Meeting"
        board_meeting = None
        for item in data_nse:
            if item.get('desc') == "Outcome of Board Meeting":
                board_meeting = item
        exchdisstime = board_meeting['exchdisstime'] if board_meeting else None
        exchdisstime_list.append(exchdisstime)

finally:
    driver.quit()

df['exchdisstime'] = exchdisstime_list

# Step 5: Print the final DataFrame
print("\nFinal DataFrame with exchdisstime:")
print(df)


https://www.nseindia.com/api/corporate-disclosure-getquote?symbol=HAL&corpType=announcement&market=equities&from_date=01-05-2025&to_date=14-05-2025
[{'symbol': 'HAL', 'desc': 'Outcome of Board Meeting', 'dt': '14052025131438', 'attchmntFile': 'https://nsearchives.nseindia.com/corporate/HAL_14052025131313_auditedfinancialresults_31032025_.pdf', 'sm_name': 'Hindustan Aeronautics Limited', 'sm_isin': 'INE066F01012', 'an_dt': '14-May-2025 13:14:38', 'sort_date': '2025-05-14 13:14:38', 'seq_id': None, 'smIndustry': None, 'orgid': None, 'attchmntText': 'Hindustan Aeronautics Limited has informed the Exchange regarding Outcome of Board Meeting held on May 14, 2025.', 'bflag': None, 'old_new': None, 'csvName': None, 'exchdisstime': '14-May-2025 13:14:38', 'difference': '00:00:00'}, {'symbol': 'HAL', 'desc': 'Analysts/Institutional Investor Meet/Con. Call Updates', 'dt': '13052025194525', 'attchmntFile': 'https://nsearchives.nseindia.com/corporate/HAL_13052025194427_ConferenceCall13052025.pdf',

In [3]:
df.to_excel('./stockanalysis/stock_analysis_results.xlsx', index=False)

In [4]:
import openpyxl